## Fraud Detection Analysis

## 1. Setup and Imports

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.stats import chi2_contingency, ks_2samp, mannwhitneyu
import warnings
import os
from io import StringIO

warnings.filterwarnings('ignore')

# Configuration
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 1000)

print("✓ Libraries imported successfully")
print("✓ Configuration set")

✓ Libraries imported successfully
✓ Configuration set


## 2. Load Data

In [2]:
# Define file paths
input_filepath = 'Raw_Data/credit_card_fraud_10k.csv'
output_filepath = 'Cleaned_Data/credit_card_fraud_10k_cleaned.csv'

# Create output directory if it doesn't exist
output_dir = os.path.dirname(output_filepath)
if output_dir and not os.path.exists(output_dir):
    os.makedirs(output_dir)
    print(f"✓ Created output directory: {output_dir}")

# Load data
print(f"Loading data from: {input_filepath}")
df = pd.read_csv(input_filepath)
print(f"✓ Loaded {len(df)} records from file")
print("="*80)
print("FRAUD DETECTION DATA PIPELINE - INITIALIZED")
print("="*80)

Loading data from: Raw_Data/credit_card_fraud_10k.csv
✓ Loaded 10000 records from file
FRAUD DETECTION DATA PIPELINE - INITIALIZED


## 3. Data Quality Assessment
### Step 1: Schema Validation

In [3]:
print("\n[STEP 1] SCHEMA VALIDATION")
print("-" * 80)

expected_schema = {
    'transaction_id': 'int64',
    'amount': 'float64',
    'transaction_hour': 'int64',
    'merchant_category': 'object',
    'foreign_transaction': 'int64',
    'location_mismatch': 'int64',
    'device_trust_score': 'int64',
    'velocity_last_24h': 'int64',
    'cardholder_age': 'int64',
    'is_fraud': 'int64'
}

schema_valid = True
for col, dtype in expected_schema.items():
    if col not in df.columns:
        print(f"  ✗ Missing column: {col}")
        schema_valid = False
    elif df[col].dtype != dtype:
        print(f"  ⚠ Column {col}: Expected {dtype}, got {df[col].dtype}")

if schema_valid:
    print("  ✓ Schema validation PASSED")


[STEP 1] SCHEMA VALIDATION
--------------------------------------------------------------------------------
  ✓ Schema validation PASSED


### Step 2: Missing Value Analysis

In [4]:
print("\n[STEP 2] MISSING VALUE ANALYSIS")
print("-" * 80)

missing_summary = pd.DataFrame({
    'Column': df.columns,
    'Missing_Count': df.isnull().sum().values,
    'Missing_Percentage': (df.isnull().sum() / len(df) * 100).values,
    'Data_Type': df.dtypes.values
})

print(missing_summary.to_string(index=False))

total_missing = df.isnull().sum().sum()
if total_missing == 0:
    print("\n  ✓ No missing values detected")
else:
    print(f"\n  ⚠ Total missing values: {total_missing}")


[STEP 2] MISSING VALUE ANALYSIS
--------------------------------------------------------------------------------
             Column  Missing_Count  Missing_Percentage Data_Type
     transaction_id              0                 0.0     int64
             amount              0                 0.0   float64
   transaction_hour              0                 0.0     int64
  merchant_category              0                 0.0    object
foreign_transaction              0                 0.0     int64
  location_mismatch              0                 0.0     int64
 device_trust_score              0                 0.0     int64
  velocity_last_24h              0                 0.0     int64
     cardholder_age              0                 0.0     int64
           is_fraud              0                 0.0     int64

  ✓ No missing values detected


### Step 3: Duplicate Detection

In [5]:
print("\n[STEP 3] DUPLICATE DETECTION")
print("-" * 80)

# Check duplicate rows
duplicate_rows = df.duplicated()
dup_count = duplicate_rows.sum()

# Check duplicate transaction IDs
dup_ids = df['transaction_id'].duplicated()
dup_id_count = dup_ids.sum()

print(f"  Duplicate Rows: {dup_count} ({dup_count/len(df)*100:.2f}%)")
print(f"  Duplicate Transaction IDs: {dup_id_count}")

if dup_count > 0:
    print("\n  Duplicate Records:")
    print(df[duplicate_rows])
else:
    print("\n  ✓ No duplicates detected")


[STEP 3] DUPLICATE DETECTION
--------------------------------------------------------------------------------
  Duplicate Rows: 0 (0.00%)
  Duplicate Transaction IDs: 0

  ✓ No duplicates detected


### Step 4: Domain Constraint Validation

In [6]:
print("\n[STEP 4] DOMAIN CONSTRAINT VALIDATION")
print("-" * 80)

validation_results = {}

# Transaction Hour: 0-23
invalid_hours = df[(df['transaction_hour'] < 0) | (df['transaction_hour'] > 23)]
validation_results['transaction_hour'] = len(invalid_hours)
print(f"  transaction_hour (0-23): {len(invalid_hours)} invalid values")

# Device Trust Score: 0-100
invalid_scores = df[(df['device_trust_score'] < 0) | (df['device_trust_score'] > 100)]
validation_results['device_trust_score'] = len(invalid_scores)
print(f"  device_trust_score (0-100): {len(invalid_scores)} invalid values")

# Amount: positive
invalid_amounts = df[df['amount'] <= 0]
validation_results['amount'] = len(invalid_amounts)
print(f"  amount (>0): {len(invalid_amounts)} invalid values")

# Velocity: non-negative
invalid_velocity = df[df['velocity_last_24h'] < 0]
validation_results['velocity'] = len(invalid_velocity)
print(f"  velocity_last_24h (≥0): {len(invalid_velocity)} invalid values")

# Age: reasonable range
invalid_age = df[(df['cardholder_age'] < 18) | (df['cardholder_age'] > 100)]
validation_results['cardholder_age'] = len(invalid_age)
print(f"  cardholder_age (18-100): {len(invalid_age)} invalid values")

# Binary fields: 0 or 1
binary_fields = ['foreign_transaction', 'location_mismatch', 'is_fraud']
for field in binary_fields:
    invalid_binary = df[~df[field].isin([0, 1])]
    validation_results[field] = len(invalid_binary)
    print(f"  {field} (0/1): {len(invalid_binary)} invalid values")

total_invalid = sum(validation_results.values())
if total_invalid == 0:
    print("\n  ✓ All domain constraints satisfied")
else:
    print(f"\n  ⚠ Total domain violations: {total_invalid}")


[STEP 4] DOMAIN CONSTRAINT VALIDATION
--------------------------------------------------------------------------------
  transaction_hour (0-23): 0 invalid values
  device_trust_score (0-100): 0 invalid values
  amount (>0): 1 invalid values
  velocity_last_24h (≥0): 0 invalid values
  cardholder_age (18-100): 0 invalid values
  foreign_transaction (0/1): 0 invalid values
  location_mismatch (0/1): 0 invalid values
  is_fraud (0/1): 0 invalid values

  ⚠ Total domain violations: 1


### Step 5: Outlier Detection

In [7]:
print("\n[STEP 5] OUTLIER DETECTION")
print("-" * 80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

outlier_summary = []

for feature in numerical_features:
    # IQR Method
    Q1 = df[feature].quantile(0.25)
    Q3 = df[feature].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    
    iqr_outliers = df[(df[feature] < lower_bound) | (df[feature] > upper_bound)]
    
    # Z-Score Method (threshold = 3)
    z_scores = np.abs(stats.zscore(df[feature]))
    z_outliers = df[z_scores > 3]
    
    outlier_summary.append({
        'Feature': feature,
        'IQR_Outliers': len(iqr_outliers),
        'Z_Score_Outliers': len(z_outliers),
        'Lower_Bound': lower_bound,
        'Upper_Bound': upper_bound,
        'Min': df[feature].min(),
        'Max': df[feature].max()
    })

outlier_df = pd.DataFrame(outlier_summary)
print(outlier_df.to_string(index=False))


[STEP 5] OUTLIER DETECTION
--------------------------------------------------------------------------------
           Feature  IQR_Outliers  Z_Score_Outliers  Lower_Bound  Upper_Bound  Min     Max
            amount           501               180    -236.4575     529.8425  0.0 1471.04
  transaction_hour             0                 0     -12.0000      36.0000  0.0   23.00
device_trust_score             0                 0     -12.5000     135.5000 25.0   99.00
 velocity_last_24h            51                51      -2.0000       6.0000  0.0    9.00
    cardholder_age             0                 0      -9.0000      95.0000 18.0   69.00


### Step 6: Statistical Profiling

In [8]:
print("\n[STEP 6] STATISTICAL PROFILING")
print("-" * 80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

stats_profile = df[numerical_features].describe(
    percentiles=[.01, .05, .25, .5, .75, .95, .99]
).T

# Add additional statistics
stats_profile['skewness'] = df[numerical_features].skew()
stats_profile['kurtosis'] = df[numerical_features].kurtosis()
stats_profile['cv'] = stats_profile['std'] / stats_profile['mean']

print("\nNumerical Features Profile:")
print(stats_profile.round(3))


[STEP 6] STATISTICAL PROFILING
--------------------------------------------------------------------------------

Numerical Features Profile:
                      count     mean      std   min     1%      5%     25%      50%     75%      95%      99%      max  skewness  kurtosis     cv
amount              10000.0  175.950  175.393   0.0   1.94   9.329  50.905  122.095  242.48  530.208  796.323  1471.04     1.919     5.118  0.997
transaction_hour    10000.0   11.593    6.923   0.0   0.00   1.000   6.000   12.000   18.00   22.000   23.000    23.00    -0.027    -1.206  0.597
device_trust_score  10000.0   61.799   21.487  25.0  25.00  28.000  43.000   62.000   80.00   96.000   99.000    99.00     0.011    -1.180  0.348
velocity_last_24h   10000.0    2.009    1.433   0.0   0.00   0.000   1.000    2.000    3.00    5.000    6.000     9.00     0.708     0.446  0.713
cardholder_age      10000.0   43.469   14.979  18.0  18.00  20.000  30.000   44.000   56.00   67.000   69.000    69.00     0.004

### Step 7: Target Variable Analysis

In [9]:
print("\n[STEP 7] TARGET VARIABLE ANALYSIS")
print("-" * 80)

fraud_dist = df['is_fraud'].value_counts().sort_index()
fraud_pct = df['is_fraud'].value_counts(normalize=True).sort_index() * 100

print("\nTarget Variable Distribution:")
print(f"  Normal (0): {fraud_dist.get(0, 0)} ({fraud_pct.get(0, 0):.2f}%)")
print(f"  Fraud (1):  {fraud_dist.get(1, 0)} ({fraud_pct.get(1, 0):.2f}%)")

if 1 in fraud_dist.index and 0 in fraud_dist.index:
    imbalance_ratio = fraud_dist[0] / fraud_dist[1]
    print(f"\n  Imbalance Ratio: {imbalance_ratio:.2f}:1")
    
    if imbalance_ratio > 10:
        print("  ⚠ HIGH IMBALANCE - Consider resampling techniques")
    elif imbalance_ratio > 5:
        print("  ⚠ MODERATE IMBALANCE - Monitor model performance")
    else:
        print("  ✓ ACCEPTABLE BALANCE")
elif 1 not in fraud_dist.index:
    print("\n  ⚠ CRITICAL: No fraud cases in dataset")


[STEP 7] TARGET VARIABLE ANALYSIS
--------------------------------------------------------------------------------

Target Variable Distribution:
  Normal (0): 9849 (98.49%)
  Fraud (1):  151 (1.51%)

  Imbalance Ratio: 65.23:1
  ⚠ HIGH IMBALANCE - Consider resampling techniques


### Step 8: Categorical Feature Analysis

In [10]:
print("\n[STEP 8] CATEGORICAL FEATURE ANALYSIS")
print("-" * 80)

# Merchant Category
merchant_dist = df['merchant_category'].value_counts()
merchant_pct = df['merchant_category'].value_counts(normalize=True) * 100

print("\nMerchant Category Distribution:")
for cat, count in merchant_dist.items():
    print(f"  {cat:15s}: {count:4d} ({merchant_pct[cat]:5.2f}%)")

print(f"\n  Total unique categories: {df['merchant_category'].nunique()}")


[STEP 8] CATEGORICAL FEATURE ANALYSIS
--------------------------------------------------------------------------------

Merchant Category Distribution:
  Food           : 2093 (20.93%)
  Clothing       : 2050 (20.50%)
  Travel         : 1990 (19.90%)
  Grocery        : 1944 (19.44%)
  Electronics    : 1923 (19.23%)

  Total unique categories: 5


### Step 9: Correlation Analysis

In [11]:
print("\n[STEP 9] CORRELATION ANALYSIS")
print("-" * 80)

numerical_cols = ['amount', 'transaction_hour', 'device_trust_score', 
                 'velocity_last_24h', 'cardholder_age', 
                 'foreign_transaction', 'location_mismatch', 'is_fraud']

corr_matrix = df[numerical_cols].corr()

print("\nCorrelation with Target Variable (is_fraud):")
target_corr = corr_matrix['is_fraud'].sort_values(ascending=False)
for feature, corr_val in target_corr.items():
    if feature != 'is_fraud':
        print(f"  {feature:25s}: {corr_val:7.4f}")

# High correlations between features (multicollinearity check)
print("\nHigh Feature Correlations (|r| > 0.7):")
high_corr_pairs = []
for i in range(len(corr_matrix.columns)):
    for j in range(i+1, len(corr_matrix.columns)):
        if abs(corr_matrix.iloc[i, j]) > 0.7:
            high_corr_pairs.append((
                corr_matrix.columns[i],
                corr_matrix.columns[j],
                corr_matrix.iloc[i, j]
            ))

if high_corr_pairs:
    for feat1, feat2, corr_val in high_corr_pairs:
        print(f"  {feat1} <-> {feat2}: {corr_val:.4f}")
else:
    print("  ✓ No multicollinearity detected")


[STEP 9] CORRELATION ANALYSIS
--------------------------------------------------------------------------------

Correlation with Target Variable (is_fraud):
  foreign_transaction      :  0.1856
  location_mismatch        :  0.1730
  velocity_last_24h        :  0.1034
  amount                   :  0.0284
  cardholder_age           : -0.0006
  device_trust_score       : -0.1379
  transaction_hour         : -0.1387

High Feature Correlations (|r| > 0.7):
  ✓ No multicollinearity detected


## 4. Data Quality Report

In [12]:
print("\n" + "="*80)
print("DATA QUALITY SCORECARD")
print("="*80)

total_records = len(df)
total_features = len(df.columns) - 1  # Excluding target

# Calculate quality score
missing_values = df.isnull().sum().sum()
duplicates = df.duplicated().sum()
domain_violations = sum(validation_results.values())

quality_scores = {
    'Completeness': 100 if missing_values == 0 else 
                  (1 - missing_values / (total_records * total_features)) * 100,
    'Uniqueness': 100 if duplicates == 0 else
                 (1 - duplicates / total_records) * 100,
    'Validity': 100 if domain_violations == 0 else 95,
}

overall_quality = np.mean(list(quality_scores.values()))

print(f"\nDataset Size: {total_records} records × {total_features + 1} features")
print(f"\nQuality Dimensions:")
for dimension, score in quality_scores.items():
    status = "✓" if score >= 95 else "⚠"
    print(f"  {status} {dimension:15s}: {score:6.2f}%")

print(f"\n{'='*40}")
print(f"  Overall Quality Score: {overall_quality:.2f}%")
print(f"{'='*40}")

# Quality grade
if overall_quality >= 95:
    grade = "EXCELLENT"
elif overall_quality >= 85:
    grade = "GOOD"
elif overall_quality >= 70:
    grade = "FAIR"
else:
    grade = "POOR"

print(f"\n  Data Quality Grade: {grade}")


DATA QUALITY SCORECARD

Dataset Size: 10000 records × 10 features

Quality Dimensions:
  ✓ Completeness   : 100.00%
  ✓ Uniqueness     : 100.00%
  ✓ Validity       :  95.00%

  Overall Quality Score: 98.33%

  Data Quality Grade: EXCELLENT


## 5. Exploratory Data Analysis
### Univariate Analysis

In [13]:
print("\n" + "="*80)
print("UNIVARIATE ANALYSIS")
print("="*80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

for feature in numerical_features:
    print(f"\n{feature.upper()}")
    print("-" * 40)
    
    # Descriptive statistics
    print(f"  Mean:       {df[feature].mean():.2f}")
    print(f"  Median:     {df[feature].median():.2f}")
    print(f"  Std Dev:    {df[feature].std():.2f}")
    print(f"  Range:      [{df[feature].min():.2f}, {df[feature].max():.2f}]")
    print(f"  IQR:        {df[feature].quantile(0.75) - df[feature].quantile(0.25):.2f}")
    
    # Distribution shape
    skew = df[feature].skew()
    kurt = df[feature].kurtosis()
    print(f"  Skewness:   {skew:.3f} {'(Right-skewed)' if skew > 0 else '(Left-skewed)' if skew < 0 else '(Symmetric)'}")
    print(f"  Kurtosis:   {kurt:.3f} {'(Heavy-tailed)' if kurt > 0 else '(Light-tailed)'}")


UNIVARIATE ANALYSIS

AMOUNT
----------------------------------------
  Mean:       175.95
  Median:     122.09
  Std Dev:    175.39
  Range:      [0.00, 1471.04]
  IQR:        191.57
  Skewness:   1.919 (Right-skewed)
  Kurtosis:   5.118 (Heavy-tailed)

TRANSACTION_HOUR
----------------------------------------
  Mean:       11.59
  Median:     12.00
  Std Dev:    6.92
  Range:      [0.00, 23.00]
  IQR:        12.00
  Skewness:   -0.027 (Left-skewed)
  Kurtosis:   -1.206 (Light-tailed)

DEVICE_TRUST_SCORE
----------------------------------------
  Mean:       61.80
  Median:     62.00
  Std Dev:    21.49
  Range:      [25.00, 99.00]
  IQR:        37.00
  Skewness:   0.011 (Right-skewed)
  Kurtosis:   -1.180 (Light-tailed)

VELOCITY_LAST_24H
----------------------------------------
  Mean:       2.01
  Median:     2.00
  Std Dev:    1.43
  Range:      [0.00, 9.00]
  IQR:        2.00
  Skewness:   0.708 (Right-skewed)
  Kurtosis:   0.446 (Heavy-tailed)

CARDHOLDER_AGE
-------------------

### Bivariate Analysis

In [14]:
print("\n" + "="*80)
print("BIVARIATE ANALYSIS (Features vs Fraud)")
print("="*80)

if df['is_fraud'].nunique() < 2:
    print("\n  ⚠ Cannot perform bivariate analysis - only one class present")
else:
    numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                         'velocity_last_24h', 'cardholder_age']
    
    for feature in numerical_features:
        print(f"\n{feature.upper()} by Fraud Status:")
        print("-" * 40)
        
        fraud_stats = df.groupby('is_fraud')[feature].describe()
        print(fraud_stats)
        
        # Statistical test (Mann-Whitney U)
        normal_vals = df[df['is_fraud'] == 0][feature]
        fraud_vals = df[df['is_fraud'] == 1][feature]
        
        if len(fraud_vals) > 0:
            stat, p_value = mannwhitneyu(normal_vals, fraud_vals)
            print(f"\n  Mann-Whitney U test p-value: {p_value:.4f}")
            if p_value < 0.05:
                print(f"  ✓ Significant difference between groups")
            else:
                print(f"  ✗ No significant difference")


BIVARIATE ANALYSIS (Features vs Fraud)

AMOUNT by Fraud Status:
----------------------------------------
           count        mean         std   min    25%     50%      75%      max
is_fraud                                                                       
0         9849.0  175.333015  173.986837  0.00  50.99  122.11  241.650  1471.04
1          151.0  216.182980  248.120467  0.11  41.53  118.94  341.695  1185.07

  Mann-Whitney U test p-value: 0.5720
  ✗ No significant difference

TRANSACTION_HOUR by Fraud Status:
----------------------------------------
           count       mean       std  min  25%   50%   75%   max
is_fraud                                                         
0         9849.0  11.712154  6.870960  0.0  6.0  12.0  18.0  23.0
1          151.0   3.841060  5.803554  0.0  1.0   2.0   3.0  23.0

  Mann-Whitney U test p-value: 0.0000
  ✓ Significant difference between groups

DEVICE_TRUST_SCORE by Fraud Status:
----------------------------------------
      

### Multivariate Pattern Analysis

In [15]:
print("\n" + "="*80)
print("MULTIVARIATE PATTERN ANALYSIS")
print("="*80)

# Amount by merchant and fraud
print("\nAverage Amount by Merchant Category:")
merchant_amount = df.groupby('merchant_category')['amount'].agg(['mean', 'std', 'count'])
print(merchant_amount.round(2))

# Risk factor combinations
print("\nRisk Factor Combinations:")
risk_analysis = df.groupby(['foreign_transaction', 'location_mismatch']).agg({
    'is_fraud': ['count', 'sum', 'mean'],
    'amount': 'mean',
    'device_trust_score': 'mean'
}).round(2)
print(risk_analysis)


MULTIVARIATE PATTERN ANALYSIS

Average Amount by Merchant Category:
                     mean     std  count
merchant_category                       
Clothing           176.02  175.64   2050
Electronics        178.61  183.51   1923
Food               173.83  169.03   2093
Grocery            176.72  177.00   1944
Travel             174.79  172.21   1990

Risk Factor Combinations:
                                      is_fraud            amount device_trust_score
                                         count sum  mean    mean               mean
foreign_transaction location_mismatch                                              
0                   0                     8250  26  0.00  175.58              61.67
                    1                      772  43  0.06  177.12              62.11
1                   0                      893  53  0.06  177.02              62.83
                    1                       85  29  0.34  189.79              60.53


### Feature Importance Analysis

In [16]:
print("\n" + "="*80)
print("FEATURE IMPORTANCE INDICATORS")
print("="*80)

numerical_features = ['amount', 'transaction_hour', 'device_trust_score', 
                     'velocity_last_24h', 'cardholder_age']

# Variance-based
print("\nFeature Variance (Higher = More Informative):")
for feature in numerical_features:
    var = df[feature].var()
    cv = df[feature].std() / df[feature].mean() if df[feature].mean() != 0 else 0
    print(f"  {feature:25s}: Variance={var:10.2f}, CV={cv:.4f}")

# Information value (if fraud cases exist)
if df['is_fraud'].sum() > 0:
    print("\nPreliminary Feature-Target Association:")
    for feature in numerical_features:
        corr = df[feature].corr(df['is_fraud'])
        print(f"  {feature:25s}: r={corr:7.4f}")


FEATURE IMPORTANCE INDICATORS

Feature Variance (Higher = More Informative):
  amount                   : Variance=  30762.64, CV=0.9968
  transaction_hour         : Variance=     47.92, CV=0.5971
  device_trust_score       : Variance=    461.69, CV=0.3477
  velocity_last_24h        : Variance=      2.05, CV=0.7131
  cardholder_age           : Variance=    224.37, CV=0.3446

Preliminary Feature-Target Association:
  amount                   : r= 0.0284
  transaction_hour         : r=-0.1387
  device_trust_score       : r=-0.1379
  velocity_last_24h        : r= 0.1034
  cardholder_age           : r=-0.0006


## 6. Executive Summary & Save Cleaned Data

In [17]:
print("\n" + "="*80)
print("EXECUTIVE SUMMARY")
print("="*80)
print(f"\n✓ Data Cleaning Completed")
print(f"✓ Exploratory Data Analysis Completed")
print(f"✓ Data Quality Score: {overall_quality:.2f}%")
print(f"✓ Ready for Feature Engineering & Modeling")
print("\n" + "="*80)

# Save cleaned data
df.to_csv(output_filepath, index=False)
print(f"\n✓ Cleaned data saved to: {output_filepath}")
print(f"✓ Total records saved: {len(df)}")


EXECUTIVE SUMMARY

✓ Data Cleaning Completed
✓ Exploratory Data Analysis Completed
✓ Data Quality Score: 98.33%
✓ Ready for Feature Engineering & Modeling


✓ Cleaned data saved to: Cleaned_Data/credit_card_fraud_10k_cleaned.csv
✓ Total records saved: 10000


## 7. Quick Data Overview

In [18]:
# Display first few rows
print("\nFirst 5 rows of cleaned data:")
df.head()


First 5 rows of cleaned data:


,transaction_id,amount,transaction_hour,merchant_category,foreign_transaction,location_mismatch,device_trust_score,velocity_last_24h,cardholder_age,is_fraud
0,1,84.47,22,Electronics,0,0,66,3,40,0
1,2,541.82,3,Travel,1,0,87,1,64,0
2,3,237.01,17,Grocery,0,0,49,1,61,0
3,4,164.33,4,Grocery,0,1,72,3,34,0
4,5,30.53,15,Food,0,0,79,0,44,0


In [19]:
# Display basic info
print("\nDataset Information:")
df.info()


Dataset Information:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   transaction_id       10000 non-null  int64  
 1   amount               10000 non-null  float64
 2   transaction_hour     10000 non-null  int64  
 3   merchant_category    10000 non-null  object 
 4   foreign_transaction  10000 non-null  int64  
 5   location_mismatch    10000 non-null  int64  
 6   device_trust_score   10000 non-null  int64  
 7   velocity_last_24h    10000 non-null  int64  
 8   cardholder_age       10000 non-null  int64  
 9   is_fraud             10000 non-null  int64  
dtypes: float64(1), int64(8), object(1)
memory usage: 781.4+ KB


In [20]:
# Display statistical summary
print("\nStatistical Summary:")
df.describe()


Statistical Summary:


,transaction_id,amount,transaction_hour,foreign_transaction,location_mismatch,device_trust_score,velocity_last_24h,cardholder_age,is_fraud
count,10000.00000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,5000.50000,175.949849,11.593300,0.097800,0.085700,61.798900,2.008900,43.468700,0.015100
std,2886.89568,175.392827,6.922708,0.297059,0.279935,21.487053,1.432559,14.979147,0.121957
min,1.00000,0.000000,0.000000,0.000000,0.000000,25.000000,0.000000,18.000000,0.000000
25%,2500.75000,50.905000,6.000000,0.000000,0.000000,43.000000,1.000000,30.000000,0.000000
50%,5000.50000,122.095000,12.000000,0.000000,0.000000,62.000000,2.000000,44.000000,0.000000
75%,7500.25000,242.480000,18.000000,0.000000,0.000000,80.000000,3.000000,56.000000,0.000000
max,10000.00000,1471.040000,23.000000,1.000000,1.000000,99.000000,9.000000,69.000000,1.000000
